In [1]:
import sys
import boto3
import json
import pandas as pd
from my_secrets import my_secrets

# Assignment 2: Part 1

In [2]:
###########################################
#                READ DATA                #
###########################################

raw_data = pd.read_csv('data/Corona_NLP_test.csv').tail(10)
mturk_data = pd.read_csv('./assignments_1-25.csv')
mturk_data['ATime'] = pd.to_datetime(mturk_data['AcceptTime'])
mturk_data['STime'] = pd.to_datetime(mturk_data['SubmitTime'])

f = open('results_1-25.json')
results = json.load(f)
res_df = pd.DataFrame.from_records(results)


In [3]:
# for res in results:
#     print(res['hit_id'] + " " + str(res['sentiment']))

res_dict = {
    "389A2A3052C7KXMI1CS5Z71AN5BC0I": 3,
    "3AJA9FLWTQSPU4N8W47CDHM96CKFIR": 5,
    "3X7837UUBRSP6ME3DTP8VCOWQS66JR": 4,
    "3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3": 1,
    "3O0M2G5VDKWMLNXG8T5G9KQFYWO94E": 2,
    "37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L": 4,
    "3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3": 2,
    "3TFJJUELTVJEG7PWGZWJIVUPK3VC22": 3,
    "3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2": 1,
    "3TL87MO8D0JRK5VEH74P5XE1NGEFL5": 5,
}

res_df['HITId'] = res_df['hit_id'] 

In [4]:
mturk_data['GroundTruth'] = mturk_data.apply(lambda x: res_dict[x['HITId']], axis=1)
mturk_data['MinDiff'] = (mturk_data.STime - mturk_data.ATime) / pd.Timedelta(minutes=1)
mturk_data['SecDiff'] = (mturk_data.STime - mturk_data.ATime) / pd.Timedelta(seconds=1)

mturk_data.head()

,AssignmentId,WorkerId,HITId,AutoApprovalTime,AcceptTime,SubmitTime,Answer,ATime,STime,GroundTruth,MinDiff,SecDiff
0,3RYC5T2D7IOG00DEQW2SSNWJMLUPRE,A2IXK982JL0ECO,389A2A3052C7KXMI1CS5Z71AN5BC0I,2023-02-24 14:21:49-05:00,2023-01-25 14:21:01-05:00,2023-01-25 14:21:49-05:00,1,2023-01-25 14:21:01-05:00,2023-01-25 14:21:49-05:00,3,0.800000,48.0
1,3SLE99ER028NYXYVH8E4J6TTEYYZBW,A2JX2VIJWXHS4X,389A2A3052C7KXMI1CS5Z71AN5BC0I,2023-02-24 14:24:33-05:00,2023-01-25 14:24:13-05:00,2023-01-25 14:24:33-05:00,4,2023-01-25 14:24:13-05:00,2023-01-25 14:24:33-05:00,3,0.333333,20.0
2,3QAPZX2QNJ8W8ZS9UL81CEN32L3206,A2JX2VIJWXHS4X,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,2023-02-24 14:26:06-05:00,2023-01-25 14:25:56-05:00,2023-01-25 14:26:06-05:00,4,2023-01-25 14:25:56-05:00,2023-01-25 14:26:06-05:00,5,0.166667,10.0
3,3JJVG1YBEQSPRJE6AK7OU2MYDZLB5W,A2CRQ0XPXN1P0K,3X7837UUBRSP6ME3DTP8VCOWQS66JR,2023-02-23 22:02:17-05:00,2023-01-24 22:02:12-05:00,2023-01-24 22:02:17-05:00,1,2023-01-24 22:02:12-05:00,2023-01-24 22:02:17-05:00,4,0.083333,5.0
4,3HMVI3QICYNMG96YJPJWZN7IMS41YP,A2JX2VIJWXHS4X,3X7837UUBRSP6ME3DTP8VCOWQS66JR,2023-02-24 14:25:38-05:00,2023-01-25 14:25:31-05:00,2023-01-25 14:25:38-05:00,4,2023-01-25 14:25:31-05:00,2023-01-25 14:25:38-05:00,4,0.116667,7.0


Based on the average of mturk responses, how accurate are the existing labels?

In [5]:
avg_df = mturk_data[['HITId', 'Answer', 'GroundTruth']]
avg_df.head(3)


,HITId,Answer,GroundTruth
0,389A2A3052C7KXMI1CS5Z71AN5BC0I,1,3
1,389A2A3052C7KXMI1CS5Z71AN5BC0I,4,3
2,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,4,5


In [6]:
agg_df = avg_df.groupby("HITId")["Answer", "GroundTruth"].agg(["mean", "std", "max", "count"])
agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')


/var/folders/60/3nzg3xnj7s54m11cbswy4sf80000gn/T/ipykernel_92965/8824565.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_df = avg_df.groupby("HITId")["Answer", "GroundTruth"].agg(["mean", "std", "max", "count"])


In [7]:
agg_df['diff'] = (agg_df['Answer|mean'] - agg_df['GroundTruth|mean']).abs()
agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "Answer|mean", "GroundTruth|mean", "diff"]].sort_values(by=['diff'], ascending=False)


,tweet,HITId,Answer|mean,GroundTruth|mean,diff
8,I and @ForestandPaper members are committed to...,3TL87MO8D0JRK5VEH74P5XE1NGEFL5,2.75,5.0,2.25
5,"@RicePolitics @MDCounties Craig, will you call...",3O0M2G5VDKWMLNXG8T5G9KQFYWO94E,3.50,2.0,1.50
6,Gov need to do somethings instead of biar je r...,3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2,2.50,1.0,1.50
0,Meanwhile In A Supermarket in Israel -- People...,37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L,3.00,4.0,1.00
2,@calebmealer @thebradfordfile @realDonaldTrump...,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,4.00,5.0,1.00
9,"Even though the Law Library is closed, ALL sub...",3X7837UUBRSP6ME3DTP8VCOWQS66JR,3.00,4.0,1.00
1,You never eaten the pigs cat dog or food from ...,389A2A3052C7KXMI1CS5Z71AN5BC0I,2.50,3.0,0.50
4,"With Gov Hogan's announcement that all bars, r...",3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3,1.50,1.0,0.50
7,Asst Prof of Economics @cconces was on @NBCPhi...,3TFJJUELTVJEG7PWGZWJIVUPK3VC22,3.50,3.0,0.50
3,Did you panic buy a lot of non-perishable item...,3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3,2.00,2.0,0.00


Which tasks had the highest STD DEV?

In [8]:
agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "diff", "Answer|std"]].sort_values(by=['Answer|std'], ascending=False)


,tweet,HITId,diff,Answer|std
1,You never eaten the pigs cat dog or food from ...,389A2A3052C7KXMI1CS5Z71AN5BC0I,0.50,2.121320
8,I and @ForestandPaper members are committed to...,3TL87MO8D0JRK5VEH74P5XE1NGEFL5,2.25,2.061553
9,"Even though the Law Library is closed, ALL sub...",3X7837UUBRSP6ME3DTP8VCOWQS66JR,1.00,1.732051
0,Meanwhile In A Supermarket in Israel -- People...,37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L,1.00,1.414214
3,Did you panic buy a lot of non-perishable item...,3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3,0.00,1.000000
5,"@RicePolitics @MDCounties Craig, will you call...",3O0M2G5VDKWMLNXG8T5G9KQFYWO94E,1.50,0.707107
6,Gov need to do somethings instead of biar je r...,3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2,1.50,0.707107
7,Asst Prof of Economics @cconces was on @NBCPhi...,3TFJJUELTVJEG7PWGZWJIVUPK3VC22,0.50,0.707107
4,"With Gov Hogan's announcement that all bars, r...",3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3,0.50,0.577350
2,@calebmealer @thebradfordfile @realDonaldTrump...,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,1.00,NaN


Which tasks took the longest? Calculated on average (in minutes)


In [9]:
avg_df = mturk_data[['HITId', 'MinDiff']]
agg_df = avg_df.groupby("HITId")["MinDiff"].agg(["mean", "std", "max", "count"])
# agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')
# agg_df.head()

agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "mean"]].sort_values(by=['mean'], ascending=False)

,tweet,HITId,mean
5,"@RicePolitics @MDCounties Craig, will you call...",3O0M2G5VDKWMLNXG8T5G9KQFYWO94E,1.208333
6,Gov need to do somethings instead of biar je r...,3O2Y2UIUD4OG0ZDWW7SOQX1MYG9FK2,1.066667
3,Did you panic buy a lot of non-perishable item...,3AXFSPQOZ4SLPF7EEMSCPI19JR9FJ3,0.722222
1,You never eaten the pigs cat dog or food from ...,389A2A3052C7KXMI1CS5Z71AN5BC0I,0.566667
4,"With Gov Hogan's announcement that all bars, r...",3J5XXLQDI05Z0YRFBG5TRXN6BOB3V3,0.458333
8,I and @ForestandPaper members are committed to...,3TL87MO8D0JRK5VEH74P5XE1NGEFL5,0.416667
0,Meanwhile In A Supermarket in Israel -- People...,37VUR2VJ7OJ7RM70XVW5DCKKSXHC1L,0.366667
7,Asst Prof of Economics @cconces was on @NBCPhi...,3TFJJUELTVJEG7PWGZWJIVUPK3VC22,0.308333
2,@calebmealer @thebradfordfile @realDonaldTrump...,3AJA9FLWTQSPU4N8W47CDHM96CKFIR,0.166667
9,"Even though the Law Library is closed, ALL sub...",3X7837UUBRSP6ME3DTP8VCOWQS66JR,0.127778


Which workers worked the fastest? And what was the variability in their answers

In [10]:
avg_df = mturk_data[['WorkerId', 'MinDiff', "Answer"]]
agg_df = avg_df.groupby("WorkerId")["MinDiff", "Answer"].agg(["mean", "std", "max", "min", "count"])
agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')
agg_df.sort_values(by=['MinDiff|mean'], ascending=False)

# agg_df.merge(res_df, how='inner', on='WorkerId')[['tweet', 'WorkerId', "mean"]].sort_values(by=['mean'], ascending=False)

/var/folders/60/3nzg3xnj7s54m11cbswy4sf80000gn/T/ipykernel_92965/2439504496.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_df = avg_df.groupby("WorkerId")["MinDiff", "Answer"].agg(["mean", "std", "max", "min", "count"])


,MinDiff|mean,MinDiff|std,MinDiff|max,MinDiff|min,MinDiff|count,Answer|mean,Answer|std,Answer|max,Answer|min,Answer|count
WorkerId,,,,,,,,,,
A27W8C25KKKFXE,1.583333,NaN,1.583333,1.583333,1,2.000000,NaN,2,2,1
AW5RT9YS268VA,1.533333,NaN,1.533333,1.533333,1,2.000000,NaN,2,2,1
A2IXK982JL0ECO,0.744444,0.386341,1.100000,0.333333,3,1.666667,1.154701,3,1,3
AOSDB00WXVME4,0.657143,0.723793,1.966667,0.033333,7,2.428571,1.397276,4,1,7
A2JX2VIJWXHS4X,0.293333,0.166704,0.550000,0.116667,10,3.300000,0.948683,4,2,10
A6EEIASZ5KBPV,0.233333,NaN,0.233333,0.233333,1,5.000000,NaN,5,5,1
A2CRQ0XPXN1P0K,0.083333,NaN,0.083333,0.083333,1,1.000000,NaN,1,1,1
A2QHKSGU21S0GA,0.050000,NaN,0.050000,0.050000,1,1.000000,NaN,1,1,1


In [11]:
# Produce final dataset


# Assignment 2: Part 2

In [43]:
###########################################
#                READ DATA                #
###########################################

raw_data = pd.read_csv('data/Corona_NLP_chosen.csv').tail(10)
mturk_data = pd.read_csv('./assignments_1-30.csv')
mturk_data['ATime'] = pd.to_datetime(mturk_data['AcceptTime'])
mturk_data['STime'] = pd.to_datetime(mturk_data['SubmitTime'])

f = open('results_1-30.json')
results = json.load(f)
res_df = pd.DataFrame.from_records(results)
res_df['HITId'] = res_df['hit_id']
res_dict = dict(zip(res_df.hit_id, res_df.sentiment))



In [44]:
def sentiment_to_numeric(x):
    if x=='Extremely Negative': return 1
    if x=='Negative': return 2
    if x=='Neutral': return 3
    if x=='Positive': return 4
    if x=='Extremely Positive': return 5

mturk_data['Sentiment'] = mturk_data['Answer'].apply(sentiment_to_numeric)
mturk_data['GroundTruth'] = mturk_data.apply(lambda x: res_dict[x['HITId']], axis=1)
mturk_data['MinDiff'] = (mturk_data.STime - mturk_data.ATime) / pd.Timedelta(minutes=1)
mturk_data['SecDiff'] = (mturk_data.STime - mturk_data.ATime) / pd.Timedelta(seconds=1)
mturk_data.head(3)



,AssignmentId,WorkerId,HITId,AutoApprovalTime,AcceptTime,SubmitTime,Answer,ATime,STime,Sentiment,GroundTruth,MinDiff,SecDiff
0,3EJJQNKU960ONJFQ7JUNJEUS0ULRHC,A2XEZ2RZUS2NMI,3OCZWXS7021ZI82PBVRGJLNOYPOL5M,2023-02-24 17:55:09-05:00,2023-01-25 17:55:01-05:00,2023-01-25 17:55:09-05:00,Neutral,2023-01-25 17:55:01-05:00,2023-01-25 17:55:09-05:00,3,3,0.133333,8.0
1,354P56DE9ZY3V99R9Z8GIBAU8T8S7Z,A2XEZ2RZUS2NMI,3IJ95K7NEB6NM5UU44QM0AI2TT2NG8,2023-02-24 18:33:18-05:00,2023-01-25 18:33:12-05:00,2023-01-25 18:33:18-05:00,Positive,2023-01-25 18:33:12-05:00,2023-01-25 18:33:18-05:00,4,5,0.100000,6.0
2,3IAEQB9FMTFCJZRAKHL3FSNB0HJWDN,A1W3VZV2POI85H,3VJ4PFXFKH1Z7410F5AYIJZT3UIUAV,2023-02-26 21:26:21-05:00,2023-01-27 21:26:08-05:00,2023-01-27 21:26:21-05:00,Negative,2023-01-27 21:26:08-05:00,2023-01-27 21:26:21-05:00,2,5,0.216667,13.0


In [51]:
# AVGs

avg_df = mturk_data[['HITId', 'Sentiment', 'GroundTruth']]
avg_df.head(3)
agg_df = avg_df.groupby("HITId")["Sentiment", "GroundTruth"].agg(["mean", "std", "max", "count"])
agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')

agg_df['diff'] = (agg_df['Sentiment|mean'] - agg_df['GroundTruth|mean']).abs()
agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "Sentiment|mean", "GroundTruth|mean", "diff", "Sentiment|count"]].sort_values(by=['diff'], ascending=False)


/var/folders/60/3nzg3xnj7s54m11cbswy4sf80000gn/T/ipykernel_92965/1475580060.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_df = avg_df.groupby("HITId")["Sentiment", "GroundTruth"].agg(["mean", "std", "max", "count"])


,tweet,HITId,Sentiment|mean,GroundTruth|mean,diff,Sentiment|count
1,Popped into my local supermarket after work. T...,38RHULDVAC9R5Q93MTHCH5891IYIWM,2.000000,4.0,2.000000,3
7,"#Covid_19 Went to the Grocery Store, turns out...",3VJ4PFXFKH1Z7410F5AYIJZT3UIUAV,3.000000,5.0,2.000000,3
2,Small reminder: stock up on bar soap for your ...,3IJ95K7NEB6NM5UU44QM0AI2TT2NG8,3.250000,5.0,1.750000,4
6,Two important phone numbers NYers should know....,3OLZC0DJ9X9EQ19YWJCGV3PVU9WIVC,3.200000,2.0,1.200000,5
0,"While you are stocking up, waiting for quarant...",3126F2F5GMXP5CAGEODQ4W2LGSOPEP,3.000000,2.0,1.000000,3
9,"Remember, with all the media deflection storie...",3WKGUBL7TDGHMPYPXQ86ETHAYTTL4H,2.000000,1.0,1.000000,3
8,The beginning of a global recession! #Coronav...,3VQTAXTYOHF4QE6SLJAUDUJZLC3UBQ,1.666667,1.0,0.666667,3
3,@Disney What would it take for my family to pa...,3JYPJ2TAZW2HBQAC9ZXWMENLST3PF7,3.666667,4.0,0.333333,3
4,"When youre stockpiling food & other supplies, ...",3MD8CKRQ0DHIS5CN5HS667FSMI8RJ4,2.666667,3.0,0.333333,3
5,Just did online shopping and first available s...,3OCZWXS7021ZI82PBVRGJLNOYPOL5M,2.833333,3.0,0.166667,6


In [48]:
# STD DEV

agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "diff", "Sentiment|std"]].sort_values(by=['Sentiment|std'], ascending=False)


,tweet,HITId,diff,Sentiment|std
0,"While you are stocking up, waiting for quarant...",3126F2F5GMXP5CAGEODQ4W2LGSOPEP,1.000000,1.000000
1,Popped into my local supermarket after work. T...,38RHULDVAC9R5Q93MTHCH5891IYIWM,2.000000,1.000000
7,"#Covid_19 Went to the Grocery Store, turns out...",3VJ4PFXFKH1Z7410F5AYIJZT3UIUAV,2.000000,1.000000
9,"Remember, with all the media deflection storie...",3WKGUBL7TDGHMPYPXQ86ETHAYTTL4H,1.000000,1.000000
2,Small reminder: stock up on bar soap for your ...,3IJ95K7NEB6NM5UU44QM0AI2TT2NG8,1.750000,0.957427
5,Just did online shopping and first available s...,3OCZWXS7021ZI82PBVRGJLNOYPOL5M,0.166667,0.752773
3,@Disney What would it take for my family to pa...,3JYPJ2TAZW2HBQAC9ZXWMENLST3PF7,0.333333,0.577350
4,"When youre stockpiling food & other supplies, ...",3MD8CKRQ0DHIS5CN5HS667FSMI8RJ4,0.333333,0.577350
8,The beginning of a global recession! #Coronav...,3VQTAXTYOHF4QE6SLJAUDUJZLC3UBQ,0.666667,0.577350
6,Two important phone numbers NYers should know....,3OLZC0DJ9X9EQ19YWJCGV3PVU9WIVC,1.200000,0.447214


In [49]:
# longest tasks

avg_df = mturk_data[['HITId', 'MinDiff']]
agg_df = avg_df.groupby("HITId")["MinDiff"].agg(["mean", "std", "max", "count"])

agg_df.merge(res_df, how='inner', on='HITId')[['tweet', 'HITId', "mean"]].sort_values(by=['mean'], ascending=False)

,tweet,HITId,mean
3,@Disney What would it take for my family to pa...,3JYPJ2TAZW2HBQAC9ZXWMENLST3PF7,0.500000
5,Just did online shopping and first available s...,3OCZWXS7021ZI82PBVRGJLNOYPOL5M,0.480556
1,Popped into my local supermarket after work. T...,38RHULDVAC9R5Q93MTHCH5891IYIWM,0.438889
0,"While you are stocking up, waiting for quarant...",3126F2F5GMXP5CAGEODQ4W2LGSOPEP,0.294444
2,Small reminder: stock up on bar soap for your ...,3IJ95K7NEB6NM5UU44QM0AI2TT2NG8,0.279167
9,"Remember, with all the media deflection storie...",3WKGUBL7TDGHMPYPXQ86ETHAYTTL4H,0.277778
6,Two important phone numbers NYers should know....,3OLZC0DJ9X9EQ19YWJCGV3PVU9WIVC,0.270000
4,"When youre stockpiling food & other supplies, ...",3MD8CKRQ0DHIS5CN5HS667FSMI8RJ4,0.261111
7,"#Covid_19 Went to the Grocery Store, turns out...",3VJ4PFXFKH1Z7410F5AYIJZT3UIUAV,0.244444
8,The beginning of a global recession! #Coronav...,3VQTAXTYOHF4QE6SLJAUDUJZLC3UBQ,0.094444


In [50]:
avg_df = mturk_data[['WorkerId', 'MinDiff', "Sentiment"]]
agg_df = avg_df.groupby("WorkerId")["MinDiff", "Sentiment"].agg(["mean", "std", "max", "min", "count"])
agg_df.columns = agg_df.columns.map('|'.join).str.strip('|')
agg_df.sort_values(by=['MinDiff|mean'], ascending=False)

/var/folders/60/3nzg3xnj7s54m11cbswy4sf80000gn/T/ipykernel_92965/704464305.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_df = avg_df.groupby("WorkerId")["MinDiff", "Sentiment"].agg(["mean", "std", "max", "min", "count"])


,MinDiff|mean,MinDiff|std,MinDiff|max,MinDiff|min,MinDiff|count,Sentiment|mean,Sentiment|std,Sentiment|max,Sentiment|min,Sentiment|count
WorkerId,,,,,,,,,,
A3F5D6DN4AMTGZ,1.366667,NaN,1.366667,1.366667,1,2.0,NaN,2,2,1
AOUQO4R0XPPYF,0.508333,0.459619,0.833333,0.183333,2,3.0,0.000000,3,3,2
AOSDB00WXVME4,0.373333,0.127947,0.566667,0.150000,10,2.7,1.059350,4,1,10
AT6U29D7VCL7L,0.300000,NaN,0.300000,0.300000,1,3.0,NaN,3,3,1
A2JX2VIJWXHS4X,0.296667,0.271006,0.983333,0.066667,10,2.6,0.966092,4,1,10
A1W3VZV2POI85H,0.206667,0.130573,0.550000,0.066667,10,2.9,0.875595,4,2,10
A2XEZ2RZUS2NMI,0.116667,0.023570,0.133333,0.100000,2,3.5,0.707107,4,3,2
